In [22]:
import json
import os
import pandas as pd
from matplotlib import pyplot as plt

data_dir = "./data"
if not os.path.exists(data_dir):
  os.makedirs(data_dir)

res_dir = "./results"
if not os.path.exists(res_dir):
  os.makedirs(res_dir)

speech_df = pd.read_csv(os.path.join(data_dir, "test.csv"))

messages = speech_df["Message"].tolist()
labels = speech_df["Aggressive"].tolist()

with open(os.path.join(res_dir, "pre_trained_pred.json"), "r") as f:
  pre_trained_pred = json.load(f)
with open(os.path.join(res_dir, "fine_tuned_pred.json"), "r") as f:
  fine_tuned_pred = json.load(f)
with open(os.path.join(res_dir, "perspective_indices.json"), "r") as f:
  perspective_indices = json.load(f)
with open(os.path.join(res_dir, "perspective_probs.json"), "r") as f:
  perspective_probs = json.load(f)
with open(os.path.join(res_dir, "perspective_pred.json"), "r") as f:
  perspective_pred = json.load(f)

print(len(pre_trained_pred), len(fine_tuned_pred), len(perspective_pred))

labels = [label for i, label in enumerate(labels) if i in perspective_indices]
pre_trained_pred = [pred for i, pred in enumerate(pre_trained_pred) if  i in perspective_indices]
fine_tuned_pred = [pred for i, pred in enumerate(fine_tuned_pred) if  i in perspective_indices]

print(len(pre_trained_pred), len(fine_tuned_pred), len(perspective_pred), len(labels))

def get_classification_result(y_true, y_pred):
  result = {"TP": [], "FP": [], "FN": [], "TN": []}
  for i, (true, pred) in enumerate(zip(y_true, y_pred)):
    if true == 1 and pred == 1:
      result["TP"].append(i)
    elif true == 1 and pred == 0:
      result["FN"].append(i)
    elif true == 0 and pred == 1:
      result["FP"].append(i)
    elif true == 0 and pred == 0:
      result["TN"].append(i)
  return result

def get_classification_metrics(classification_result):
  metrics = {}

  true_pos = len(classification_result["TP"])
  false_pos = len(classification_result["FP"])
  false_neg = len(classification_result["FN"])
  true_neg = len(classification_result["TN"])

  metrics["tp"] = true_pos
  metrics["fp"] = false_pos
  metrics["fn"] = false_neg
  metrics["tn"] = true_neg
  metrics["fpr"] = false_pos / (false_pos + true_neg)
  metrics["fnr"] = false_neg / (false_neg + true_pos)
  metrics["accuracy"] = (true_pos + true_neg) / (true_pos + false_pos + false_neg + true_neg)
  metrics["recall"] = true_pos / (true_pos + false_neg)
  metrics["precision"] = true_pos / (true_pos + false_pos)

  return metrics

pretrain_result = get_classification_result(labels, pre_trained_pred)
fine_tuned_result = get_classification_result(labels, fine_tuned_pred)
perspective_result = get_classification_result(labels, perspective_pred)

pretrain_metrics = get_classification_metrics(pretrain_result)
fine_tuned_metrics = get_classification_metrics(fine_tuned_result)
perspective_metrics = get_classification_metrics(perspective_result)

print("Pre-trained Model")
print(pretrain_metrics)

print("Fine-tuned Model")
print(fine_tuned_metrics)

print("Perspective API")
print(perspective_metrics)

6421 6421 6370
6370 6370 6370 6370
Pre-trained Model
{'tp': 1015, 'fp': 2929, 'fn': 692, 'tn': 1734, 'fpr': 0.6281363928801201, 'fnr': 0.4053895723491506, 'accuracy': 0.43155416012558867, 'recall': 0.5946104276508495, 'precision': 0.25735294117647056}
Fine-tuned Model
{'tp': 1398, 'fp': 330, 'fn': 309, 'tn': 4333, 'fpr': 0.07076989062835085, 'fnr': 0.18101933216168717, 'accuracy': 0.8996860282574568, 'recall': 0.8189806678383128, 'precision': 0.8090277777777778}
Perspective API
{'tp': 969, 'fp': 169, 'fn': 738, 'tn': 4494, 'fpr': 0.036242762170276645, 'fnr': 0.43233743409490333, 'accuracy': 0.8576138147566719, 'recall': 0.5676625659050967, 'precision': 0.851493848857645}


In [31]:
threshold = 0.6

ensembled_pred = [max(1 if prob > threshold else 0, pred) for prob, pred in zip(perspective_probs, fine_tuned_pred)]

ensembled_result = get_classification_result(labels, ensembled_pred)
ensembled_metrics = get_classification_metrics(ensembled_result)

print("Ensembled Model")
print(ensembled_metrics)

Ensembled Model
{'tp': 1432, 'fp': 345, 'fn': 275, 'tn': 4318, 'fpr': 0.07398670383873043, 'fnr': 0.1611013473930873, 'accuracy': 0.902668759811617, 'recall': 0.8388986526069128, 'precision': 0.8058525604952167}


In [36]:
import random

random.seed(256)

fine_tuned_fp = set(fine_tuned_result["FP"])
perspective_fp = set(perspective_result["FP"])
ensembled_fp = set(ensembled_result["FP"])

all_fp = list(fine_tuned_fp & perspective_fp & ensembled_fp)
all_fp = random.sample(all_fp, len(all_fp))

print(len(all_fp))

fine_tuned_fn = set(fine_tuned_result["FN"])
perspective_fn = set(perspective_result["FN"])
ensembled_fn = set(ensembled_result["FN"])

all_fn = list(fine_tuned_fn & perspective_fn & ensembled_fn)
all_fn = random.sample(all_fn, len(all_fn))

print(len(all_fn))

128
245


In [42]:
k = 10

print("False Positive Samples")
for i in all_fp[:k]:
  print(f'\t {messages[perspective_indices[i]]}')

print("False Negative Samples")
for i in all_fn[:k]:
  print(f'\t {messages[perspective_indices[i]]}')

False Positive Samples
	 Oi twat why are you editing my segments Todd Howard is a twat hence why I said so
	 boring prick    get off bobby zamoras page and get outdoors
	 You will often find that Xenocidic just talks bollocks Utter bollocks
	 I agree with myself on this matter and alas Ive yet to find one pity A fuckin MEN Some kind of new hot garbage idk Not better than slim shady Curious about them and all of you dont get me wrong i hate them they all deserve to die and burn or the other way around but from what i hear god is omnipotent what makes you think you can speak for himherit whatever you get the point only a fag would say that YOURE GOING TO HELL FAG thats when you know youre going to hell And to whoever asked about a guitarist check with Bop Skizzum i think he got back with them
	 Nooo massive forehead stays damn it MKR
	 There arent many girl alter servers though Why is it that everyone on youtube likes Ron Paul yet Romney has won both Iowa and New HampshireWTF How does th